In [395]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams, FreqDist
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn import feature_extraction, linear_model, model_selection, preprocessing



/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [58]:
train_df  = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [59]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [60]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [61]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [62]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [63]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [64]:
test_df.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

need to take care to all the null but not to forget the amount of the nulls in the test_df

In [65]:
first_sent_example = train_df.text[0].lower()
first_sent_example

'our deeds are the reason of this #earthquake may allah forgive us all'

In [66]:
stop_words=set(stopwords.words('english'))

In [76]:
tokens_first= word_tokenize(first_sent_example)
print(tokens_first)

['our', 'deeds', 'are', 'the', 'reason', 'of', 'this', '#', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all']


In [68]:
filtered_words = [w for w in tokens_first if w not in stop_words]
filtered_words

['deeds', 'reason', '#', 'earthquake', 'may', 'allah', 'forgive', 'us']

In [69]:
wordnet_lemmatizer = WordNetLemmatizer()
lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in filtered_words]
lemmatized_word

['deed', 'reason', '#', 'earthquake', 'may', 'allah', 'forgive', 'u']

In [70]:
print(nltk.pos_tag(lemmatized_word))

[('deed', 'VB'), ('reason', 'NN'), ('#', '#'), ('earthquake', 'NN'), ('may', 'MD'), ('allah', 'VB'), ('forgive', 'JJ'), ('u', 'NN')]


In [71]:
only_words = [word for word in lemmatized_word if word.isalpha()]
only_words

['deed', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'u']

In [75]:
anthoer_sent_example = train_df.text[15].lower()
anthoer_sent_example

"what's up man?"

this is not a disaster tweets

In [78]:
tokens_not_disaster= word_tokenize(anthoer_sent_example)
print(tokens_not_disaster)

['what', "'s", 'up', 'man', '?']


In [79]:
filtered_words_not_disaster = [w for w in tokens_not_disaster if w not in stop_words]
filtered_words_not_disaster

["'s", 'man', '?']

In [80]:
lemmatized_word_not_disaster = [wordnet_lemmatizer.lemmatize(word) for word in filtered_words_not_disaster]
lemmatized_word_not_disaster

["'s", 'man', '?']

In [81]:
only_words_not_disaster = [word for word in lemmatized_word_not_disaster if word.isalpha()]
only_words_not_disaster

['man']

clearly it's not a disaster

In [93]:
anthoer_sent_example_2 = train_df.text[32].lower()
anthoer_sent_example_2

'we always try to bring the heavy. #metal #rt http://t.co/yao1e0xngw'

In [94]:
tokens_not_disaster_2= word_tokenize(anthoer_sent_example_2)
print(tokens_not_disaster_2)

['we', 'always', 'try', 'to', 'bring', 'the', 'heavy', '.', '#', 'metal', '#', 'rt', 'http', ':', '//t.co/yao1e0xngw']


In [96]:
filtered_words_not_disaster_2 = [w for w in tokens_not_disaster_2 if w not in stop_words]
print(filtered_words_not_disaster_2)

['always', 'try', 'bring', 'heavy', '.', '#', 'metal', '#', 'rt', 'http', ':', '//t.co/yao1e0xngw']


In [98]:
lemmatized_word_not_disaster_2 = [wordnet_lemmatizer.lemmatize(word) for word in filtered_words_not_disaster_2]
print(lemmatized_word_not_disaster_2)

['always', 'try', 'bring', 'heavy', '.', '#', 'metal', '#', 'rt', 'http', ':', '//t.co/yao1e0xngw']


In [99]:
only_words_not_disaster_2 = [word for word in lemmatized_word_not_disaster_2 if word.isalpha()]
only_words_not_disaster_2

['always', 'try', 'bring', 'heavy', 'metal', 'rt', 'http']

In [100]:
print(nltk.pos_tag(only_words_not_disaster_2))

[('always', 'RB'), ('try', 'VB'), ('bring', 'VBG'), ('heavy', 'JJ'), ('metal', 'NN'), ('rt', 'NN'), ('http', 'NN')]


so i found out those 3 sentence and need to get better disintc this is after cleaning:

1) ['deed', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'u']

2) ['man']

3) ['always', 'try', 'bring', 'heavy', 'metal', 'rt', 'http']


















so how can i get the one words like earthquake and not "man" to get the context:
first step is to suggest if the sentence is without "NN" it's not a disaster tweets in some probality.

In [159]:
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_df.text)
test_vectors = vectorizer.transform(test_df.text)

In [160]:
print(train_vectors)

  (0, 14003)	1
  (0, 5490)	1
  (0, 2192)	1
  (0, 18669)	1
  (0, 15678)	1
  (0, 13681)	1
  (0, 18777)	1
  (0, 6379)	1
  (0, 12141)	1
  (0, 1852)	1
  (0, 7661)	1
  (0, 19774)	1
  (0, 1851)	1
  (1, 7652)	1
  (1, 7439)	1
  (1, 13122)	1
  (1, 11091)	1
  (1, 16266)	1
  (1, 16611)	1
  (1, 3843)	1
  (2, 2192)	2
  (2, 1851)	1
  (2, 15940)	1
  (2, 2312)	1
  (2, 18971)	1
  :	:
  (7611, 11518)	1
  (7611, 16895)	1
  (7611, 1670)	1
  (7611, 9967)	1
  (7611, 18814)	1
  (7611, 13361)	1
  (7611, 3047)	2
  (7611, 4589)	1
  (7611, 14783)	1
  (7611, 18088)	1
  (7611, 9841)	1
  (7612, 18669)	1
  (7612, 3698)	1
  (7612, 3797)	1
  (7612, 9304)	1
  (7612, 4517)	1
  (7612, 13195)	1
  (7612, 12698)	1
  (7612, 11187)	1
  (7612, 13391)	1
  (7612, 1434)	1
  (7612, 20606)	1
  (7612, 9170)	1
  (7612, 15606)	1
  (7612, 21262)	1


In [161]:
vectorizer_tfid = TfidfVectorizer()
vectorizer_tfid.fit(train_df.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [179]:
print(vectorizer_tfid.vocabulary_)
print(vectorizer_tfid.idf_)
# encode document
vector = vectorizer_tfid.transform([train_df.text[1]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

{'our': 14003, 'deeds': 5490, 'are': 2192, 'the': 18669, 'reason': 15678, 'of': 13681, 'this': 18777, 'earthquake': 6379, 'may': 12141, 'allah': 1852, 'forgive': 7661, 'us': 19774, 'all': 1851, 'forest': 7652, 'fire': 7439, 'near': 13122, 'la': 11091, 'ronge': 16266, 'sask': 16611, 'canada': 3843, 'residents': 15940, 'asked': 2312, 'to': 18971, 'shelter': 17022, 'in': 9718, 'place': 14612, 'being': 2900, 'notified': 13423, 'by': 3698, 'officers': 13701, 'no': 13335, 'other': 13987, 'evacuation': 6909, 'or': 13919, 'orders': 13936, 'expected': 7014, '13': 176, '000': 1, 'people': 14389, 'receive': 15699, 'wildfires': 20607, 'california': 3797, 'just': 10550, 'got': 8364, 'sent': 16870, 'photo': 14500, 'from': 7823, 'ruby': 16379, 'alaska': 1800, 'as': 2280, 'smoke': 17394, 'pours': 14821, 'into': 9947, 'school': 16700, 'rockyfire': 16219, 'update': 19722, 'hwy': 9399, '20': 343, 'closed': 4462, 'both': 3346, 'directions': 5819, 'due': 6248, 'lake': 11118, 'county': 4915, 'cafire': 3768,

In [317]:
clean_tex = []
PorterS = PorterStemmer()
for i in range(train_df.text.shape[0]):
    without_stop_words = [PorterS.stem(word.lower()) for word in train_df.text.str.split()[i] if not word  in stop_words  if word.isalpha()]
    ff = ' '.join(without_stop_words)
    clean_tex.append(ff)

In [320]:
print(pd.DataFrame(clean_tex)[0].head(5))

0                  our deed reason may allah forgiv us
1                      forest fire near la rong canada
2    all resid ask notifi no evacu shelter place or...
3                  peopl receiv evacu order california
4           just got sent photo rubi smoke pour school
Name: 0, dtype: object


In [403]:
vectorizer_tfid_after_cleaning = TfidfVectorizer()
vectorizer_tfid_after_cleaning.fit(clean_tex)
print(vectorizer_tfid_after_cleaning)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [404]:
print(vectorizer_tfid_after_cleaning.vocabulary_)

{'our': 5461, 'deed': 1892, 'reason': 6191, 'may': 4693, 'allah': 199, 'forgiv': 2877, 'us': 8095, 'forest': 2872, 'fire': 2765, 'near': 5127, 'la': 4230, 'rong': 6466, 'canada': 1114, 'all': 198, 'resid': 6324, 'ask': 422, 'notifi': 5252, 'no': 5208, 'evacu': 2515, 'shelter': 6818, 'place': 5768, 'order': 5434, 'expect': 2568, 'peopl': 5660, 'receiv': 6202, 'california': 1091, 'just': 4072, 'got': 3178, 'sent': 6732, 'photo': 5713, 'rubi': 6514, 'smoke': 6996, 'pour': 5880, 'school': 6655, 'updat': 8070, 'close': 1404, 'direct': 2048, 'due': 2244, 'lake': 4245, 'counti': 1651, 'heavi': 3440, 'rain': 6125, 'caus': 1195, 'flash': 2795, 'flood': 2820, 'street': 7300, 'colorado': 1480, 'spring': 7174, 'area': 369, 'top': 7768, 'hill': 3499, 'see': 6704, 'emerg': 2387, 'happen': 3356, 'build': 1009, 'across': 61, 'afraid': 109, 'tornado': 7775, 'come': 1490, 'three': 7672, 'die': 2027, 'heat': 3436, 'wave': 8315, 'far': 2648, 'haha': 3313, 'south': 7098, 'tampa': 7520, 'get': 3085, 'wait':

ok nice , so now we have all the words counting inside a dict from the text

In [405]:
dict_count = vectorizer_tfid_after_cleaning.vocabulary_

d_xtrain = dict(list(dict_count.items())[len(dict_count)//2:])
d_ytrain = dict(list(dict_count.items())[:len(dict_count)//2])
